# Magnitudes of high-redshift targets

### Robert Quimby

This notebook gives an example of how I am calculating magnitudes and colors for high-redshift sources.

Procedure outline:
- load a spectral template
- rescale the flux densities of the template to a distance of 10pc
- redshift the wavelengths and flux densities to the desired redshift
- convolve the predicted spectra with a filter response curve to calculate the AB magnitude

The heart of this procedure is the `redshift_spec` function below. 

In [1]:
import numpy as np
import speclite
import speclite.filters
from scipy.interpolate import interp1d
import astropy.units as u
from astropy.cosmology import WMAP9 as cosmo

In [2]:
# load Hsiao SNIa templates
# (these can be downloaded from http://astrophysics.physics.fsu.edu/~hsiao/data/)
hsiao = np.genfromtxt('/home/jovyan/research/Current/hsiao_templates/snflux_1a.dat', names='phase, wav, flux')

# pick data at B-band max
wmax = hsiao['phase'] == 0
template = hsiao[wmax]

In [3]:
def scale_spec(data, band, mag):
    """
    rescale flux values to give the desired `mag` in some `band`
    """
    
    flux, wlen = band.pad_spectrum(data['flux'], data['wav'])
    mag0 = band.get_ab_magnitude(flux, wlen)
    scale = 10**(-0.4 * (mag - mag0))
    data['flux'] *= scale

In [4]:
# scale to some absolute magnitude in some band
absmag = -19.3
rband = speclite.filters.load_filter('sdss2010-r')
scale_spec(template, rband, absmag)

In [5]:
# verify the scaling worked
flux, wlen = rband.pad_spectrum(template['flux'], template['wav'])
rband.get_ab_magnitude(flux, wlen)

-19.3

In [6]:
def redshift_spec(rest, z):
    """
    calculate the flux densities of a `rest` spectrum redshifted to `z`
    
    `rest` is a numpy structured array. 
    `rest['wav']` holds the rest frame wavelengths
    `rest['flux']` holds the flux density (flam) at 10pc
    """

    # convert to luminosity
    flam = u.erg / u.second / u.cm**2 / u.angstrom
    llam = u.erg / u.second / u.angstrom
    flux = rest['flux'] * flam
    lum = (4 * np.pi * flux * (10 * u.pc)**2).to(llam)

    # redshift
    observed = rest.copy()
    observed['wav'] = rest['wav'] * (1 + z)
    # following eqn. 23 in Hogg (1999/2000) http://adsabs.harvard.edu/abs/1999astro.ph..5116H
    slam = (1 / (1 + z)) * lum / (4 * np.pi * cosmo.luminosity_distance(z)**2)
    observed['flux'] = slam.to(flam)

    return observed

In [8]:
z =0.05
observed = redshift_spec(template, z)
flux, wlen = rband.pad_spectrum(observed['flux'], observed['wav'])
rband.get_ab_magnitude(flux, wlen)

17.37528346323216

In [2]:
def compute_observed_magnitude(phase, z, filter_choice):
    import numpy as np
    import speclite
    import speclite.filters
    import astropy.units as u
    from astropy.cosmology import WMAP9 as cosmo

    # Nested helper function to scale the spectrum
    def scale_spec(data, band, mag):
        """
        Rescale the flux values so that the synthetic AB magnitude 
        computed through the given filter equals the desired magnitude.
        """
        flux, wlen = band.pad_spectrum(data['flux'], data['wav'])
        mag0 = band.get_ab_magnitude(flux, wlen)
        scale = 10 ** (-0.4 * (mag - mag0))
        data['flux'] *= scale

    # Nested helper function to redshift the spectrum
    def redshift_spec(rest, z):
        """
        Redshift a rest-frame spectrum.
        """
        flam = u.erg / u.second / u.cm**2 / u.angstrom
        llam = u.erg / u.second / u.angstrom
        # Convert flux at 10pc to a luminosity
        flux = rest['flux'] * flam
        lum = (4 * np.pi * flux * (10 * u.pc) ** 2).to(llam)
        observed = rest.copy()
        observed['wav'] = rest['wav'] * (1 + z)
        # Adjust flux using the luminosity distance correction
        slam = (1 / (1 + z)) * lum / (4 * np.pi * cosmo.luminosity_distance(z) ** 2)
        observed['flux'] = slam.to(flam)
        return observed

    # Define constants
    absmag = -19.3  # Absolute magnitude used to scale the template
    hsiao_file = "/home/jovyan/research/Current/hsiao_templates/snflux_1a.dat"

    # Load the Hsiao SNIa template data
    hsiao = np.genfromtxt(hsiao_file, names='phase, wav, flux')

    # Select the template rows for the specified phase (or the closest match)
    mask = hsiao['phase'] == phase
    if not np.any(mask):
        idx = np.argmin(np.abs(hsiao['phase'] - phase))
        closest_phase = hsiao['phase'][idx]
        print(f"Phase {phase} not found. Using closest available phase: {closest_phase:.1f}")
        mask = hsiao['phase'] == closest_phase
    template = hsiao[mask]

    # Load the filter based on the user's choice ('r' or 'g')
    filter_choice = filter_choice.lower()
    if filter_choice == 'r':
        filt = speclite.filters.load_filter('sdss2010-r')
    elif filter_choice == 'g':
        filt = speclite.filters.load_filter('sdss2010-g')
    elif filter_choice == 'b':
        filt = speclite.filters.load_filter('sdss2010-b')
    else:
        raise ValueError("Invalid filter choice. Please choose 'r' 'b' 'g'.")

    # Scale the template to the desired absolute magnitude
    scale_spec(template, filt, absmag)
    # Redshift the template spectrum
    observed = redshift_spec(template, z)
    # Compute the observed magnitude using synthetic photometry
    flux, wlen = filt.pad_spectrum(observed['flux'], observed['wav'])
    obs_mag = filt.get_ab_magnitude(flux, wlen)
    return obs_mag

In [5]:
compute_observed_magnitude(0, 0.069, 'r')

18.089947550321686

In [ ]:
17.375389144596465